In [246]:
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy.core.defchararray as np_f

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [247]:
Capital = 1000000  # Первоначальный капитал
t = 22*13 # Количество временных интервалов для расчетов (22*13 - это 1 месяц). То есть в данном случае алгоритм сичтает, 
          # что заранее известны только данные за месяц, на основе их анализа он принимает решения. 
          # Данные за остальное время изначально считаются неизвестными, на них демонстрируется работоспособность алгоритма.
T = 100  #Период, по которому аппроксимируется спред
tax = 0.0005 # Доля которую удерживает брокер при сделке
k_min = 0.9
k_max = 1
Stavka = 6,4 #Безрисковая ставка. Облигации РФ

#1 Вариант
#t_start = pd.to_datetime('2021-11-01 00:00:00')
#t_finish = pd.to_datetime('2022-06-01 00:00:00')
#metod = 'srednee' # 'srednee' или 'mnk'

#2 Вариант
#t_start = pd.to_datetime('2021-11-01 00:00:00')
#t_finish = pd.to_datetime('2022-06-01 00:00:00')
#metod = 'mnk' # 'srednee' или 'mnk'

#3 Вариант
#t_start = pd.to_datetime('2020-10-01 00:00:00')
#t_finish = pd.to_datetime('2021-11-01 00:00:00')
#metod = 'srednee' # 'srednee' или 'mnk'

#4 Вариант
# t_start = pd.to_datetime('2021-11-01 00:00:00')
# t_finish = pd.to_datetime('2022-06-01 00:00:00')

t_start = 20201001
t_finish = 20211101
metod = 'mnk' # 'srednee' или 'mnk'

In [248]:
data_start = np.zeros((2000, 11000))
name_share = dict()

for i, filename in enumerate(os.listdir("/content/drive/MyDrive/МосБиржа акции_01_01_2020_01_05_2022")):
   with open(os.path.join("/content/drive/MyDrive/МосБиржа акции_01_01_2020_01_05_2022", filename), 'r') as f:
      reader = csv.reader(f, delimiter = ",")
      for j, row in enumerate(reader):
        if j == 0:
          data_start[i+1][j] = i+1
        else:
          data_start[i+1][j] = float(row[4])
        if i == 0 and j != 0:   
          data_start[i][j] = int(row[2])
        name_share[i+1] = filename

print(data_start)

data_dt = data_start[:, (data_start[0,:] >= t_start) & (data_start[0,:] <= t_finish)].copy()
non_zero_data = np.any(data_dt != 0, axis = 1)

data = np.column_stack((data_start[:,0], data_dt))
data = data[non_zero_data, :].copy()

print(len(data[1]))
# for i in range(1,len(data)):
#   data[i,0] = data[i,data[i]>0][0]
#   for j in range(1,len(data[i])):
#    if data[i,j] == 0:
#      data[i,j] = data[i,j-1]

[[0.0000000e+00 2.0200103e+07 2.0200103e+07 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 2.2595000e+00 2.2585000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.0000000e+00 1.3204000e+04 1.3196000e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
2494


In [249]:
# pd_frame = None
# data_dir = "/content/drive/MyDrive/МосБиржа акции_01_01_2019_15_06_2022"
# for i, filename in enumerate(os.listdir(data_dir)):
#   try:
#     # считывает csv 
#     tframe = pd.read_csv(os.path.join(data_dir, filename), delimiter=',')
#   except:
#     # если не читается, то идем к следующему файлу
#     continue
#   #print(filename)
#   # Расчленяем <DATE> и <TIME> на отдельные составляющие даты и времени.
#   tframe['day'] = tframe['<DATE>'] % 100
#   tframe['month'] = (tframe['<DATE>'] // 100) % 100
#   tframe['year'] = tframe['<DATE>'] // 100_00
#   tframe['hour'] = tframe['<TIME>'] // 100_00

#   # объединями вычлененные части с использованием типа данных datetime
#   tframe['dt'] = pd.to_datetime(tframe[['year', 'month', 'day', 'hour']])

#   # переименовывам колонку с ценой открытия в название тикера.
#   TICKER = tframe['<TICKER>'].values[0]+str(i)
#   tframe = tframe.rename(columns = {'<OPEN>' : TICKER}).set_index('dt')[[TICKER]]

#   # делаем join (если датасет пустой то вместо джона простое присваивание)
#   if pd_frame is None:
#     pd_frame = tframe
#   else:
#     pd_frame = pd_frame.join(tframe)

# # заполняем дыры предшествующими значениями.
# pd_frame.fillna(method='ffill', axis=0, inplace=True)

# # выделяем нужный срез времени.
# pd_frame = pd_frame[pd_frame.index.to_series().between(t_start, t_finish)]

# # сохраняем список тикеров.
# tickers = list(pd_frame.columns.values)

# # ===== Привродим данные к тому виду, как нужно для работы программы =====
# # добавляем колонку с датой-временем в виде дейсвительного числа. 
# tmp = np_f.replace(pd_frame.index.values.astype(str), '-', '')
# tmp = np_f.replace(tmp, ':', '')
# tmp = np_f.replace(tmp, 'T', '').astype(np.float64)/(1e6)
# pd_frame['dtfloat'] = tmp


# # ======== Переходим от датафрейма к numpy массиву.
# data = np.ndarray((pd_frame.shape[0]+1, pd_frame.shape[1]))
# data[0,:] = np.arange(data.shape[1])
# data[1:,:] = pd_frame[['dtfloat']+tickers].values
# data = data.T

In [250]:
data.shape

(396, 2494)

In [251]:
def corel(first_share, second_share):
  Delta_GR = 0
  Delta_G2 = 0
  Delta_R2 = 0
  mean_first_share = sum(first_share)/len(first_share)
  mean_second_share = sum(second_share)/len(second_share)

  for i in range(len(first_share)):
      Delta_GR = Delta_GR + (first_share[i]- mean_first_share)*(second_share[i]- mean_second_share)
      Delta_G2 = Delta_G2 + (first_share[i]- mean_first_share)**2
      Delta_R2 = Delta_R2 + (second_share[i]- mean_second_share)**2

  cor = Delta_GR/((Delta_G2*Delta_R2)**0.5)
  return (cor)

In [252]:
# Метод наименьших квадратов. Полином пятой степени.

def MNK(X, Y, max_pow=8):
  A = np.vstack([np.array(X)**i for i in range(max_pow+1)]).T
  Norm = np.linalg.pinv(A).dot(Y)
  return A @ Norm

In [253]:
# Вычисление среднего спреда

def mean_spred(A, norm_B, t):
  sum_spred = 0
  for i in range(t):
    sum_spred = A[i] - norm_B[i]
  return(sum_spred / t)

In [254]:
def main_prib(first_share, second_share, time, t, tax, flag):
  first_share_norm = MNK(first_share[0:t], second_share[0:t], 1)
  ind = 0
  Sdel = []
  N_r, N_g = 0, 0
  Prib = Capital
  Stoim = Capital
  First_price = 0
  Second_price = 0
  Activ_share = 0
  
  if second_share[t-1]-first_share_norm[t-1] < 0:
    ind = 1
  for i in range(t,len(second_share)):
      first_share_norm = MNK(first_share[i-t:i], second_share[i-t:i], 1)
      Spred = second_share[i-T:i] - first_share_norm[t-T:t]
      if flag == 'mnk':
        Spred_approx = MNK(time[i-T:i], Spred[0:T], 5)
      else:
        Spred_approx = np.mean(Spred)
      if second_share[i] > Second_price + 2*Stoim*tax // first_share[i]\
        and Spred[T-1] > 0 and ind == 0 and abs(Spred[T-1]) > (second_share[i]+first_share[i])*2*tax\
        and ((flag == 'mnk' and Spred_approx[len(Spred_approx)-2] > Spred_approx[len(Spred_approx)-1])\
        or (flag == 'srednee' and Spred[T-1] > Spred_approx)):
          Prib = Prib + N_r*second_share[i]*(1-tax)
          N_r = 0
          N_g = Prib//(first_share[i]*(1+tax))
          Prib = Prib - N_g*first_share[i]*(1+tax)
          Stoim = Prib + N_g*first_share[i]*(1-tax)
          First_price = first_share[i]
          ind = 1
          Activ_share = 1  # Совершена сделка с арбитражом
      elif first_share[i] > First_price + 2*Stoim*tax//second_share[i] \
        and Spred[T-1]<0 and ind == 1 and abs(Spred[T-1]) > (second_share[i]+first_share[i])*2*tax\
        and ((flag == 'mnk' and Spred_approx[len(Spred_approx)-2] < Spred_approx[len(Spred_approx)-1])\
        or (flag == 'srednee' and Spred[T-1] > Spred_approx)):
          Prib = Prib + N_g*first_share[i]*(1-tax)
          N_g = 0
          N_r = Prib//second_share[i]*(1+tax)
          Prib = Prib - N_r*second_share[i]*(1+tax)
          Stoim = Prib + N_r*second_share[i]*(1-tax)
          Second_price = second_share[i]
          ind = 0
          Activ_share = 1  # Совершена сделка с арбитражом

  return Stoim, Activ_share


In [255]:
from tqdm import tqdm
import statistics

In [ ]:
Final_stoim = []
Prib_null = []
D_prib = []

for i in tqdm(range(1,len(data))):
  Prib_null_first = 0
  Prib_null_second = 0
  Dobavochnay_prib = 0
  for j in range(i+1,len(data)):
    Kor_d = corel(data[i][:t], data[j][:t])
    if (Kor_d >= k_min) and (Kor_d <= k_max) and (Kor_d != 1):
      S, Activ_share = main_prib(data[i], data[j], np.arange(0,data.shape[1]), t, tax, metod)
      if Activ_share == 1:
        Prib_final = (S - Capital)/Capital*100
        Final_stoim.append(Prib_final)
        Prib_null_first = ((data[i,len(data[i])-1]- data[i,t])/data[i,t])*100
        Prib_null_second = ((data[j,len(data[j])-1]- data[j,t])/data[j,t])*100
        Prib_null.append((Prib_null_second + Prib_null_first)/2)
        Dobavochnay_prib = (Prib_final - Prib_null[len(Prib_null)-1])
        #print("Первая акция : ", name_share[data[i][0]], "Доходность без арбитража : ", Prib_null_first)
        #print("Вторая акция : ", name_share[data[j][0]], "Доходность без арбитража : ", Prib_null_second)
        #print("С арбитражом : ", (S - Capital)/Capital*100)
        #print("Добавочная прибыль : ", Dobavochnay_prib)
        D_prib.append(Dobavochnay_prib)

Count_minus = 0
Count_ten = 0

for i in range(len(Final_stoim)):
    if Final_stoim[i] < 0:
      Count_minus = Count_minus + 1
    if Final_stoim[i] < 10:
      Count_ten = Count_ten + 1

Sum_prib = statistics.mean(Final_stoim)

print("Средняя доходность без арбитража :", np.mean(data[1:,-1]/data[1:,1] - 1)*100)
print("Средняя доходность : ", Sum_prib)
print("Индекс Шарпа : ", (Sum_prib - Stavka)/np.sqrt(np.var(Final_stoim)))
print("Медианное значение : ", statistics.median(Final_stoim))
print("Минимальное значение : ", min(Final_stoim))
print("Максимальное значение : ", max(Final_stoim))
print("Количество пар с отрицательной доходность : ", Count_minus)
print("Количество пар с доходностью < 10% : ", Count_ten)
print("Общее количество пар : ", len(Final_stoim))
print("Средняя добавочная доходность :", np.mean(Prib_null))

 52%|█████▏    | 204/395 [59:23<1:58:11, 37.13s/it]

# **Вывод отдельной пары**


In [ ]:
first_share = []
second_share = []
Axis = []

with open('/content/drive/MyDrive/МосБиржа топ_01_04_2020_01_04_2022/сбербанк-п_sberp_2_1_moex-akcii_sberbank-pref.csv', newline='') as File:
    reader = csv.reader(File, delimiter = ",")
    count = 0
    for row in reader:
      if count != 0 and int(row[2]) <= 20211101 and int(row[2]) >= 20201101:
        first_share.append(float(row[4]))
      count = count + 1
        
with open('/content/drive/MyDrive/МосБиржа топ_01_04_2020_01_04_2022/сбербанк_sber_2_1_moex-akcii_sberbank.csv', newline='') as File:
    reader = csv.reader(File, delimiter = ",")
    count = 0
    for row in reader:
      if count != 0 and int(row[2]) <= 20211101 and int(row[2]) >= 20201101:
        second_share.append(float(row[4]))
        Axis.append(row[2])
      count = count + 1

print(first_share)
print(second_share)

In [ ]:
  first_share_norm = MNK(first_share[0:t], second_share[0:t], 1)
  ind = 0
  Sdel = []
  N_r, N_g = 0, 0
  Prib = Capital
  Stoim = Capital
  First_price = 0
  Second_price = 0
  Activ_share = 0
  flag = metod
  Axis1 = [i for i in range(len(first_share))]
  time = Axis1
  
  if second_share[t-1]-first_share_norm[t-1] < 0:
    ind = 1
  for i in range(t,len(second_share)):
      first_share_norm = MNK(first_share[i-t:i], second_share[i-t:i], 1)
      Spred = second_share[i-T:i] - first_share_norm[t-T:t]
      if flag == 'mnk':
        Spred_approx = MNK(time[i-T:i], Spred[0:T], 5)
      else:
        Spred_approx = np.mean(Spred)
      if second_share[i] > Second_price + 2*Stoim*tax // first_share[i]\
        and Spred[T-1] > 0 and ind == 0 and abs(Spred[T-1]) > (second_share[i]+first_share[i])*2*tax\
        and ((flag == 'mnk' and Spred_approx[len(Spred_approx)-2] > Spred_approx[len(Spred_approx)-1])\
        or (flag == 'srednee' and Spred[T-1] > Spred_approx)):
          Prib = Prib + N_r*second_share[i]*(1-tax)
          N_r = 0
          N_g = Prib//(first_share[i]*(1+tax))
          Prib = Prib - N_g*first_share[i]*(1+tax)
          Stoim = Prib + N_g*first_share[i]*(1-tax)
          First_price = first_share[i]
          ind = 1
          Activ_share = 1  # Совершена сделка с арбитражом
          Sdel.append(i)
      elif first_share[i] > First_price + 2*Stoim*tax//second_share[i] \
        and Spred[T-1]<0 and ind == 1 and abs(Spred[T-1]) > (second_share[i]+first_share[i])*2*tax\
        and ((flag == 'mnk' and Spred_approx[len(Spred_approx)-2] < Spred_approx[len(Spred_approx)-1])\
        or (flag == 'srednee' and Spred[T-1] > Spred_approx)):
          Prib = Prib + N_g*first_share[i]*(1-tax)
          N_g = 0
          N_r = Prib//second_share[i]*(1+tax)
          Prib = Prib - N_r*second_share[i]*(1+tax)
          Stoim = Prib + N_r*second_share[i]*(1-tax)
          Second_price = second_share[i]
          ind = 0
          Activ_share = 1  # Совершена сделка с арбитражом
          Sdel.append(i)


print('Стоимость портфеля', Prib + N_g * first_share[len(first_share)-1] * (1-tax) + N_r*second_share[len(second_share)-1]*0.9995)
Prib = (Prib + N_g * first_share[len(first_share)-1] * (1-tax) + N_r*second_share[len(second_share)-1]*(1-tax)-Capital)/Capital*100
print('Доходность портфеля при арбитраже', Prib, '%')
print('Доходность без арбитража', (-second_share[t]+second_share[len(first_share)-1])/second_share[t+1]*100, '%')

# строим график
plt.figure(figsize=(10, 5))
S = 200
F = 500
St = 0
Fin = 2
plt.plot(Axis1[S:F],second_share[S:F], 'r--', label = 'first_share')
plt.plot(Axis1[S:F],MNK(first_share[S:F], second_share[S:F], 1), 'b--', label = 'second_share')
M = Sdel
for i in range(St, Fin):
    plt.axvline(x=M[i], color='green')
plt.legend()
plt.show()

In [ ]:
Spred = second_share[S:F] - MNK(first_share[S:F], second_share[S:F])

plt.plot(Axis1[S:F],Spred, 1, 'b--', label = 'second_share')
M = Sdel
for i in range(St, Fin):
    plt.axvline(x=M[i], color='green')